In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_excel('/content/Updated criminal Original 1 labelled.xlsx')

In [3]:
null= df.isnull()
null.sum()

Case Number             0
Accused                 0
Case Type               0
Case Description        0
Potential Punishment    0
Applicable Acts         0
Severity Score          0
dtype: int64

In [4]:
import joblib
from transformers import BertTokenizer
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer, LabelEncoder
def tokenize_text(text):
    # Replace this with the appropriate BERT model name
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    tokens = tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=128)
    return tokens

# Data Cleaning and Transformation for Textual Columns
text_columns = ['Case Description', 'Potential Punishment', 'Applicable Acts']
for column in text_columns:
    # Extract relevant information from text columns
    df[column] = df[column].apply(lambda x: tokenize_text(x))

# Label Encoding for 'Property Type' column
label_encoder = LabelEncoder()
df['Case Type'] = label_encoder.fit_transform(df['Case Type'])
joblib.dump(label_encoder, 'label_encoder.pkl')
# Apply MultiLabelBinarizer for 'Evidence' column
#mlb = MultiLabelBinarizer()
#evidence_encoded = mlb.fit_transform(df['Evidence'].apply(lambda x: x.split(',')))
#evidence_df = pd.DataFrame(evidence_encoded, columns=mlb.classes_)
#df = pd.concat([df, evidence_df], axis=1)



# Display the processed dataset
print("\nProcessed Dataset:")
print(df)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]


Processed Dataset:
     Case Number  Accused  Case Type  \
0              1    Mr. F        200   
1              2    Ms. G         11   
2              3    Mr. H        119   
3              4    Ms. I         91   
4              5    Mr. J         15   
..           ...      ...        ...   
305          306  Ms. PPP         35   
306          307  Mr. QQQ        176   
307          308  Ms. RRR         99   
308          309  Mr. SSS        211   
309          310  Ms. TTT         73   

                                      Case Description  \
0                  [101, 11065, 1037, 4684, 3042, 102]   
1            [101, 19477, 2619, 1999, 1996, 2227, 102]   
2            [101, 16789, 2619, 2041, 1997, 2769, 102]   
3                        [101, 18840, 6206, 5850, 102]   
4    [101, 5378, 1037, 26470, 2000, 1037, 2270, 794...   
..                                                 ...   
305  [101, 22817, 1998, 5500, 9385, 2098, 2678, 239...   
306  [101, 17800, 15826, 1998, 5500

In [5]:
df.drop(columns=["Accused","Case Number"], inplace=True)

In [6]:
df

,Case Type,Case Description,Potential Punishment,Applicable Acts,Severity Score
0,200,"[101, 11065, 1037, 4684, 3042, 102]","[101, 10219, 2005, 2039, 2000, 1017, 2086, 101...","[101, 12997, 2278, 2930, 4261, 2683, 102]",4
1,11,"[101, 19477, 2619, 1999, 1996, 2227, 102]","[101, 10219, 2005, 2039, 2000, 1017, 2086, 101...","[101, 12997, 2278, 2930, 25392, 102]",4
2,119,"[101, 16789, 2619, 2041, 1997, 2769, 102]","[101, 10219, 2005, 2039, 2000, 1021, 2086, 101...","[101, 12997, 2278, 2930, 17442, 102]",3
3,91,"[101, 18840, 6206, 5850, 102]","[101, 10219, 2005, 2039, 2000, 2184, 2086, 101...","[101, 6583, 29566, 4588, 5850, 1998, 18224, 13...",2
4,15,"[101, 5378, 1037, 26470, 2000, 1037, 2270, 794...","[101, 10219, 2005, 2039, 2000, 1021, 2086, 101...","[101, 9740, 1997, 7897, 2552, 1010, 2997, 102]",3
...,...,...,...,...,...
305,35,"[101, 22817, 1998, 5500, 9385, 2098, 2678, 239...","[101, 10219, 2005, 2039, 2000, 1017, 2086, 101...","[101, 9385, 2552, 102]",4
306,176,"[101, 17800, 15826, 1998, 5500, 9385, 2098, 26...","[101, 10219, 2005, 2039, 2000, 1017, 2086, 101...","[101, 9385, 2552, 102]",4
307,99,"[101, 3140, 2208, 9797, 2000, 2147, 1999, 1557...","[101, 10219, 2005, 2039, 2000, 1017, 2086, 101...","[101, 4450, 4277, 102]",4
308,211,"[101, 5117, 1999, 11703, 22048, 6078, 2066, 88...","[101, 10219, 2005, 2039, 2000, 1017, 2086, 101...","[101, 7325, 3860, 2552, 102]",4


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
import joblib



# Extracting features and target variable
features = df[['Case Description', 'Case Type', 'Potential Punishment', 'Applicable Acts']].astype(str).apply(' '.join, axis=1).tolist()
target = df['Severity Score'].tolist()

# Re-labeling classes by subtracting 1 from all class labels
target = [label - 1 for label in target]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Pipeline with CountVectorizer and XGBoost Classifier
pipeline = make_pipeline(CountVectorizer(), XGBClassifier())

# Define the parameters for GridSearchCV
param_grid = {
    'xgbclassifier__max_depth': [3, 4, 5],  # Maximum depth of trees
    'xgbclassifier__learning_rate': [0.1, 0.01, 0.05],  # Learning rate
    'xgbclassifier__n_estimators': [100, 200, 300]  # Number of trees (boosting rounds)
}

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Best parameters found by GridSearchCV
print("Best parameters:", grid_search.best_params_)

# Use the best estimator from GridSearchCV to predict
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

# Save the best model as a joblib file
joblib.dump(best_model, 'best_model.joblib')


Best parameters: {'xgbclassifier__learning_rate': 0.01, 'xgbclassifier__max_depth': 3, 'xgbclassifier__n_estimators': 200}
Accuracy: 0.7419354838709677


['best_model.joblib']

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
import pickle  # Import the pickle module


# Extracting features and target variable
features = df[['Case Description', 'Case Type', 'Potential Punishment', 'Applicable Acts']].astype(str).apply(' '.join, axis=1).tolist()
target = df['Severity Score'].tolist()

# Re-labeling classes by subtracting 1 from all class labels
target = [label - 1 for label in target]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Pipeline with CountVectorizer and XGBoost Classifier
pipeline = make_pipeline(CountVectorizer(), XGBClassifier())

# Define the parameters for GridSearchCV
param_grid = {
    'xgbclassifier__max_depth': [3, 4, 5],  # Maximum depth of trees
    'xgbclassifier__learning_rate': [0.1, 0.01, 0.05],  # Learning rate
    'xgbclassifier__n_estimators': [100, 200, 300]  # Number of trees (boosting rounds)
}

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Best parameters found by GridSearchCV
print("Best parameters:", grid_search.best_params_)

# Use the best estimator from GridSearchCV to predict
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

# Save the best model as a .pkl file using pickle
with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)


Best parameters: {'xgbclassifier__learning_rate': 0.01, 'xgbclassifier__max_depth': 3, 'xgbclassifier__n_estimators': 200}
Accuracy: 0.7419354838709677


In [11]:
import pandas as pd
from transformers import BertTokenizer
import joblib

# Define the function for text tokenization
def tokenize_text(text):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    tokens = tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=128)
    return tokens

# Define the function for preprocessing text columns
def preprocess_text_columns(df):
    text_columns = ['Charges', 'Facts', 'Potential Punishment', 'Applicable Acts']
    for column in text_columns:
        df[column] = df[column].apply(lambda x: tokenize_text(x))
    return df

# Your DataFrame creation
new_sample = pd.DataFrame({
    'Charges': ['Theft'],  # Replace with the actual value
    'Facts': ['Stealing a mobile phone'],  # Replace with the actual value
    'Potential Punishment': ['Imprisonment for up to 3 years, or fine, or both'],  # Replace with the actual value
    'Applicable Acts': ['IPC Section 379']  # Replace with the actual value
})

# Preprocess text columns
new_sample = preprocess_text_columns(new_sample)

# Load the XGBoost model
classifier = joblib.load('best_model.pkl')  # Replace 'Xg_best.joblib' with the correct path if needed

# Predict severity score using the loaded model
predicted_severity_score = classifier.predict(new_sample)

print(f'Predicted Severity Score: {predicted_severity_score[0]}')


Predicted Severity Score: 4
